# See how much data require for unigrams with local context 10 to converge

In [27]:
len(dev)

853

In [ ]:
for nrows in [200, 400, 600, 853]:
    

# Build dev set, sample5400, figure out how much data have processed

In [4]:
import pandas as pd

data = pd.read_pickle('/usr0/home/mamille2/erebor/fanfiction-project/data/features/relationship_prediction_sample6k.pkl')
print(data.columns)
print(len(data))

Index(['fic_id', 'pairing', 'pairing_embedding_embs',
       'pairing_embedding_unigrams', 'relationship', 'selected_relationships',
       'is_romantic', 'is_canon', 'is_mm'],
      dtype='object')
6000


In [5]:
# Read in fic_ids where have pipeline assertions
import os

pipeline_output_dirpath = '/usr0/home/mamille2/erebor/fanfiction-project/data/ao3/harrypotter/emnlp_dataset_6k/output'
fic_ids_processed = [int(fname.split('.')[0]) for fname in os.listdir(os.path.join(pipeline_output_dirpath, 'char_coref_chars'))]
len(fic_ids_processed)

5275

In [6]:
processed_data = data[data['fic_id'].isin(fic_ids_processed)]
len(processed_data)

5653

In [11]:
# Pairing distribution
pairing_counts = processed_data['pairing'].value_counts().to_dict()
pairing_counts

{('ginny', 'harry'): 950,
 ('hermione', 'ron'): 943,
 ('harry', 'ron'): 942,
 ('harry', 'hermione'): 941,
 ('draco', 'harry'): 939,
 ('draco', 'hermione'): 938}

In [23]:
# Take dev set out to make rest of the data even among pairs
dev = pd.DataFrame(columns=processed_data.columns)

for pairing in pairing_counts:
    
    n_sample = pairing_counts[pairing] - 800
    pairing_rows = processed_data[processed_data['pairing']==pairing].sample(n_sample)

    dev = pd.concat([dev, pairing_rows])
    
print(len(dev))
dev.head()

853


,fic_id,pairing,pairing_embedding_embs,pairing_embedding_unigrams,relationship,selected_relationships,is_romantic,is_canon,is_mm
60769,3893437,"(ginny, harry)","[-0.09474506, -0.6967747, 1.1392572, 0.3700963...","(0, 72347)\t0.0813570378252241\n (0, 97978)...","[""Draco Malfoy/Harry Potter""]","[(draco, harry)]",False,False,False
104380,9891458,"(ginny, harry)","[-0.0667619, -0.68298775, 1.2705771, 0.5642065...","(0, 97978)\t0.03231275649171685\n (0, 16108...",[],[],False,False,False
30026,13850373,"(ginny, harry)","[-0.010620714, -0.45096874, 1.2573825, 0.40351...","(0, 97978)\t0.13840863754590726\n (0, 31084...","[""Hermione Granger/Tom Riddle""]",[],False,False,False
85025,7447762,"(ginny, harry)","[0.06345814, -0.6884086, 0.7959029, 0.5834352,...","(0, 284058)\t0.08375701179585597\n (0, 5852...","[""Draco Malfoy/Harry Potter""]","[(draco, harry)]",False,False,False
5680,10397631,"(ginny, harry)","[-0.11702293, -0.5292652, 1.3536028, 0.5892232...","(0, 271840)\t0.01335013246605058\n (0, 2777...","[""Harry Potter/OMC"", ""Remus Lupin/Harry Potter...",[],False,False,False


In [24]:
nondev_data = processed_data[~processed_data.index.isin(dev.index)]
print(len(nondev_data))
nondev_data['pairing'].value_counts()

4800


(harry, hermione)    800
(draco, harry)       800
(ginny, harry)       800
(harry, ron)         800
(draco, hermione)    800
(hermione, ron)      800
Name: pairing, dtype: int64

In [25]:
dev['pairing'].value_counts()

(ginny, harry)       150
(hermione, ron)      143
(harry, ron)         142
(harry, hermione)    141
(draco, harry)       139
(draco, hermione)    138
Name: pairing, dtype: int64

In [26]:
# Save
nondev_data.to_pickle(f'/usr0/home/mamille2/erebor/fanfiction-project/data/features/relationship_prediction_sample{len(nondev_data)}.pkl')
dev.to_pickle('/usr0/home/mamille2/erebor/fanfiction-project/data/features/relationship_prediction_6k_dev.pkl')